In [2]:
import pandas as pd

## Reading DATA
- We used 7 days’s samples as training samples (20170506-20170512), and the last day’s samples as test samples (20170513).
    - 근데 까보니까 raw_sample 05-05 부터임 

In [26]:
# 유저별 광고 노출 및 클릭 여부 데이터 
raw_sample = pd.read_csv('raw_sample.csv')

# raw sample 데이터 상에서 노출된 광고들의 광고 정보 
ad_feature = pd.read_csv('raw_data/ad_feature.csv')

# 유저 정보 
user = pd.read_csv('user_profile.csv')

# 쇼핑몰 내 행동 데이터 
log = pd.read_csv('behavior_log.csv')
# This data set covers the shopping behavior in 22 days of all users in raw_sample(totally seven hundred million records 

#### Checking simple compatibility 

In [3]:
# checking compatibility 
display(raw_sample['user'].nunique() == user.shape[0] , raw_sample['adgroup_id'].nunique() == ad_feature.shape[0]) 

False

True

In [4]:
# user info 반영해서 raw_sample 을 잘라야 함 
display(raw_sample['user'].nunique(),user.shape[0])

1141729

1061768

# Preprocessing 1 

## Raw_sample Preprocessing
- nonclk 삭제 
- timestamp 변환

In [5]:
raw_sample.columns

Index(['user', 'time_stamp', 'adgroup_id', 'pid', 'nonclk', 'clk'], dtype='object')

- nonclk drop

In [6]:
#오류체크 
#sum(sum([raw_sample['nonclk'] + raw_sample['clk'] == 1]))

raw_sample = raw_sample.drop(['nonclk'],axis=1)

- time_stamp 

In [7]:
#  UNIX 시간 
# 1970년 1월 1일부터 몇초가 흘렀는지로 체크
# datetime 으로 변경  

raw_sample['time_stamp'] = pd.to_datetime(raw_sample['time_stamp'], unit='s')

In [ ]:
# user 별 time stamp sorting 

raw_sample = raw_sample.sort_values(by=['user', 'time_stamp'])

#### raw_sample preprocessing 완료

In [119]:
raw_sample.to_csv('raw_ppd_0508.csv')

## ad_feature
- customer ID 삭제

In [25]:
ad_feature = ad_feature.drop(['customer'],axis = 1)

* 결측치 존재 

In [70]:
ad_feature.isnull().sum()

adgroup_id          0
cate_id             0
campaign_id         0
customer            0
brand          246330
price               0
dtype: int64

## User_profile
- 분류그룹 획일화 
- `final_gender_code` → 이름을 gender로 변경
- `occupation`
- `new_user_class_level`
- `p_value_level`

- 분류그룹 획일화

In [82]:
# 데이터셋 설명 봐도 나눈 기준이 모호 
# 사후 분석의 용이성 , 모델 성능 (GRL) 향상의 측면에서 대분류만 남김 
print('소분류 그룹 개수:',user['cms_segid'].nunique()) 
print('대분류 그룹 개수:',user['cms_group_id'].nunique()) 

소분류 그룹 개수: 97
대분류 그룹 개수: 13


In [83]:
user = user.drop(['cms_segid'],axis=1)

- final_gender_code column 이름 변경 
- 대학생 여부 : occupation column drop (94% 가 '0' - sparse)
- 거주 지역 : new_user_class_level drop (Nan값 너무 많음 30% 이상)

In [86]:
user.rename(columns = {'final_gender_code' : 'gender'}, inplace = True)

In [103]:
user = user.drop(['occupation','new_user_class_level '],axis=1)

- pvalue_level 은 중요한 값임 , Nan 값이 50% 이상이라도 drop 하기는 아쉬운 column 
    - 측정이 안되어있다는 건 그만큼 구매가 적어서 Consumption grade 를 책정하지 못한 것으로 판단, grade 를 0 으로 설정 

In [105]:
user['pvalue_level'].value_counts(dropna=False,normalize=True)

NaN    0.542413
2.0    0.276573
1.0    0.145452
3.0    0.035562
Name: pvalue_level, dtype: float64

In [106]:
user['pvalue_level'].fillna(0,inplace=True)

### User preprocessing 완료 

In [26]:
user.columns

Index(['Unnamed: 0', 'userid', 'cms_group_id', 'gender', 'age_level',
       'pvalue_level', 'shopping_level'],
      dtype='object')

In [107]:
user.to_csv('user_ppd_0508.csv')

## behavior_log

- raw_sample 과 log 의 user 1:1 매칭 확인 - no 
- 왜냐하면, raw_sample 의 seq가 항상 conversion 으로 끝나지 않기 때문 
- behavior_log 는 쇼핑몰 내 행동 데이터로 click 이후의 내용

In [113]:
log.user.nunique() == raw_sample.user.nunique()

False

In [114]:
display(raw_sample.user.nunique(),log.user.nunique())

1141729

1136340

- time_stamp 

In [5]:
log['time_stamp'] = pd.to_datetime(log['time_stamp'], unit='s') 

In [27]:
#Kernel stopped 
#log = log.sort_values(by=['user', 'time_stamp'])

# preprocessing 2 

- user_profile에 없는 userid의 raw_sample 데이터 지울 것
- behavior_log 제외할 것 (현재는 우리 모델에 필요 X)
    - behavior_log의 **btag=buy**만 가지고 와서 사용 (adpk의 purchase 취급)
- 총 8일치 데이터 → 7일은 train, 마지막 1일을 test
- 광고 노출 등의 대해서 이상하게 값이 많은 사람 삭제하기

#### user_profile에 없는 userid의 raw_sample 데이터 지우기

In [4]:
# reading preprocessed CSVs 
user = pd.read_csv('preprocessing/user_ppd_0508.csv')
raw_sample = pd.read_csv('preprocessing/raw_ppd_0508.csv')

In [5]:
valid_userids = user['userid'].tolist()

raw_sample_filtered = raw_sample[raw_sample['user'].isin(valid_userids)]

In [6]:
print('데이터 수 : ',raw_sample.shape[0], '-->' ,raw_sample_filtered.shape[0])

데이터 수 :  26557961 --> 25029435


#### behavior_log의 **btag=buy**만 가지고 와서 사용 (adpk의 purchase 취급)

In [ ]:
# reading preprocessed CSV 
#log = pd.read_csv('raw_data/behavior_log.csv')

In [12]:
# log btag types 
log.btag.unique()

array(['buy'], dtype=object)

In [13]:
# filtering Purchases 
purchase_log = log[log.btag == 'buy']

In [15]:
# user 별 시간대 정렬 및 datetime 변환 
purchase_log = purchase_log.sort_values(by=['user', 'time_stamp'])
purchase_log['time_stamp'] = pd.to_datetime(purchase_log['time_stamp'], unit='s')  

In [20]:
# 5월 5일 부터의 노출 데이터 
raw_sample_filtered['time_stamp'].sort_values()

5314954     2017-05-05 16:00:00
5314947     2017-05-05 16:00:00
5314948     2017-05-05 16:00:00
5314949     2017-05-05 16:00:00
5314950     2017-05-05 16:00:00
                   ...         
11475487    2017-05-13 15:59:44
420843      2017-05-13 15:59:44
420844      2017-05-13 15:59:44
5704516     2017-05-13 15:59:46
4127426     2017-05-13 15:59:46
Name: time_stamp, Length: 25029435, dtype: object

In [18]:
# 4월 24일 부터의 구매 데이터 
purchase_log['time_stamp']

61224177    2017-04-24 15:00:20
531106542   2017-04-24 15:07:46
591186701   2017-04-27 02:40:06
102600864   2017-04-27 02:44:32
580113767   2017-04-27 02:52:05
                    ...        
16013059    2017-05-01 13:05:17
48139297    2017-05-01 13:06:38
270329726   2017-05-09 15:31:07
270329725   2017-05-09 15:32:38
27144852    2017-05-12 13:09:17
Name: time_stamp, Length: 9115919, dtype: datetime64[ns]

In [27]:
raw_sample_filtered.time_stamp.min()

'2017-05-05 16:00:00'

In [28]:
# purchase_log filtering 
# 광고 노출 데이터의 시간으로 purchase log 필터링 
given_time = pd.to_datetime(raw_sample_filtered.time_stamp.min())

purchase_log = purchase_log[purchase_log['time_stamp'] >= given_time ]

In [31]:
# # purchase log saving 
# purchase_log.to_csv('purchase_log_0508.csv')

### Removing Outliers
- 구매여정 길이가 너무 길거나 너무 짧으면 제거 
    - IQR method 사용 

In [14]:
seq = raw_sample_filtered.groupby('user')['user'].agg(seq_len = 'count').reset_index()

In [33]:
pd.options.display.float_format = '{:.2f}'.format
seq['seq_len'].describe()

count   1061768.00
mean         23.57
std          65.81
min           1.00
25%           3.00
50%           7.00
75%          21.00
max        3756.00
Name: seq_len, dtype: float64

In [16]:
def IQR(x):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return seq[(x >= lower_bound) & (x <= upper_bound)] #hard coding

seq = IQR(seq['seq_len'])


In [17]:
valid_userids = seq['user'].tolist()

exposed = raw_sample_filtered[raw_sample_filtered['user'].isin(valid_userids)].drop(['Unnamed: 0'],axis = 1)

In [23]:
exposed = pd.merge(exposed,seq)

#### ads feature 붙이기 
- Exposed 된 ad 에 해당하는 ad feature 를 exposed df 에 붙이기 

In [27]:
ad_feature.isnull().sum()

adgroup_id          0
cate_id             0
campaign_id         0
customer            0
brand          246330
price               0
dtype: int64

In [28]:
exposed_feat = pd.merge(exposed, ad_feature, on='adgroup_id', how='left')

In [30]:
exposed_feat.to_csv('preprocessing/exposed_log_0517.csv')